<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-географических-данных" data-toc-modified-id="Получение-географических-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Получение географических данных</a></span><ul class="toc-item"><li><span><a href="#С-помощью-API-OSM-получаем-координаты-округов" data-toc-modified-id="С-помощью-API-OSM-получаем-координаты-округов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>С помощью API OSM получаем координаты округов</a></span></li><li><span><a href="#Поиск-координат-с-помощью-Яндекс-Геокодер" data-toc-modified-id="Поиск-координат-с-помощью-Яндекс-Геокодер-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Поиск координат с помощью Яндекс Геокодер</a></span></li></ul></li><li><span><a href="#Примечание" data-toc-modified-id="Примечание-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Примечание</a></span></li></ul></div>

# Визуализация показателя больниц на географической карте

## Получение географических данных

In [15]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from geojson import Feature, Polygon, FeatureCollection
import geojson
import requests

Загружаем список городских и управленческих округов

In [13]:
districts = pd.read_excel('districts.xlsx').dropna()

In [14]:
districts

,district,unit
0,Муниципальное образование город Алапаевск,Восточный
1,Муниципальное образование Алапаевское,Восточный
2,Арамильский городской округ,Южный
3,Артёмовский городской округ,Восточный
4,Артинский городской округ,Западный
...,...,...
63,Талицкий городской округ,Восточный
64,Тугулымский городской округ,Восточный
65,Туринский городской округ,Восточный
66,Городской округ ЗАТО Уральский,Южный


### С помощью API OSM получаем координаты округов

In [10]:
url = "https://nominatim.openstreetmap.org/search?"
# Создаем словарь для будующих объектов geojson
lib_json = []
# В цикле проводим поиск через сервис nominatim по названиям округов
for j in districts.district.unique():
# param - параметры для API запроса
        param = {'q': f'{j}, Свердловская область',
        'format': 'json',
        'polygon_geojson': '1'}
    req = requests.get(url=url, params=param) 
    json = req.json()
# Если округ не обнаружен - выводим об этом сообщение    
    if len(json) == 0:
        print(f'Нет в базе {j}')
    else:
# Создаем объект Feature с координатами округа. Один округ - исключение обрабатываем иначе        
        if j != 'Горноуральский городской округ':
            lib_json.append(Feature(geometry=geojson.Polygon(req.json()[0]['geojson']['coordinates'][0])
                                  , properties={'id':len(lib_json), 'name':j}))
        else:
            print(j)
            lib_json.append(Feature(geometry=geojson.Polygon(req.json()[0]['geojson']['coordinates'])
                                  , properties={'id':len(lib_json), 'name':j}))

Нет в базе Муниципальное образование город Алапаевск
Нет в базе Муниципальное образование Алапаевское
Горноуральский городской округ
Нет в базе Муниципальное образование «город Екатеринбург»
Нет в базе Муниципальное образование город Ирбит
Нет в базе Городской округ «Город Лесной»
Нет в базе Городской округ ЗАТО Свободный
Нет в базе Городской округ ЗАТО Уральский


Создаем объект FeatureCollection с координатами всех округов

In [16]:
geo_so = FeatureCollection(lib_json)

In [18]:
with open('districts_so.geojson', 'w') as f:
    geojson.dump(geo_so, f)

### Поиск координат с помощью Яндекс Геокодер

Определяем функцию для получения координат больниц по адресу

In [19]:
# Ваш API ключ для сервиса Яндекс Геокодер
apikey = ''

In [ ]:
def get_coordinates(row: str, url='https://geocode-maps.yandex.ru/1.x?', apikey=apikey):
    '''
    Getting coordinates from API Yandex.Geolocator to adresses of hospitals of Sverdlovskaya oblast

    Input:
    row of pandas DataFrame(sample: from lambda function with axis=1), containing column: "oid" with unique code of hospital
    and column 'adress' with adress of hospital

    Output:
    Row with initial columns plus string type columns 'longitude' and 'latitude' with coordinates

    Raising:
    In case of errors in getting coordinates function prints message and continue to run
    Make new list with adress and OID of hospital with error in searching coordinates


    '''
    global adress_errors
    row['longitude'] = np.NaN
    row['latitude'] = np.NaN
    raw_adress = row.adress
    adress = raw_adress.strip().replace(' ', '+').replace('.', '+').replace(',',
                                                                            '+').replace('++', '+').replace('+++', '+')
    param = {'apikey': apikey,
             'geocode': adress,
             'rspn': '1',
             'bbox': '56.008,56.783~63.075,67.939',
             'format': 'json'
             }
    k = 0
    adresses = frmo[frmo.oid == row.oid].adress.unique()
    while k < len(adresses):
        try:
            if k == 0:
                req_coordinates = requests.get(url=url, params=param)
                row['longitude'] = (req_coordinates.json()['response']['GeoObjectCollection']['featureMember'][0]
                                    ['GeoObject']['Point']['pos'].split(' ')[0])
                row['latitude'] = (req_coordinates.json()['response']['GeoObjectCollection']['featureMember'][0]
                                   ['GeoObject']['Point']['pos'].split(' ')[1])
            else:
                adress = adresses[k-1].replace(' ', '+').replace('.', '+').replace(
                    ',', '+').replace('++', '+').replace('+++', '+')
                param = {'apikey': apikey,
                         'geocode': adress,
                         'rspn': '1',
                         'bbox': '56.008,56.783~63.075,67.939',
                         'format': 'json'
                         }
                req_coordinates = requests.get(url=url_geocoder, params=param)
                row['longitude'] = (req_coordinates.json()['response']['GeoObjectCollection']['featureMember'][0]
                                    ['GeoObject']['Point']['pos'].split(' ')[0])
                row['latitude'] = (req_coordinates.json()['response']['GeoObjectCollection']['featureMember'][0]
                                   ['GeoObject']['Point']['pos'].split(' ')[1])
        except:
            print(
                f'Ошибка при поиске координат для адреса: {adress}, OID: {row.oid}')
            adress_errors.append([raw_adress, row.oid])
            k = k + 1
        else:
            break

    return row

In [ ]:
geo_so = FeatureCollection(lib_json)

Для получения адрессов больниц(амбулаторно-поликлинического звена) используем базу из ФРМО. К сожалению не могу выложить ее в открытый доступ

In [ ]:
frmo = pd.read_excel('frmo_august.xlsx', header=6).iloc[:, [1, 2, 3, 16, 21]]
frmo.columns = ['oid', 'hospital', 'tvsp', 'adress', 'type_sp']
frmo

In [ ]:
frmo_ambulatory = frmo[frmo.type_sp.isin(['Фельдшерско-акушерские пункты (включая передвижные)', 'Поликлиники (поликлинические отделения)',
                                          'Амбулатории', 'Женские консультации', 'Стоматологические', 'Детские поликлиники (отделения, кабинеты)',
                                          'Стоматологические детские', 'Психиатрические', 'Амбулатории (включая передвижные)',
                                          'Фельдшерские пункты (включая передвижные)'])]

In [ ]:
excluded = ['ФАП', 'ОВП', 'пгт', 'филиал', 'СОШ', ' сад ', 'Фельдшерско',
            'Фельдшерско', 'ФП', 'Фап', 'фап', 'школы', 'сад"', 'Общежитие']
frmo_one_mo = frmo.query(
    "~tvsp.str.contains('|'.join(@excluded))", engine='python')
frmo_one_mo = frmo_one_mo.drop_duplicates('oid').reset_index(drop=True)

Получаем координаты больниц

In [ ]:
adress_errors = list()
frmo_one_mo = frmo_one_mo.apply(lambda x: get_coordinates(x), axis=1)

In [ ]:
frmo_one_mo.to_excel('frmo_coordinates.xlsx', index=0)

## Примечание

К сожалению точность Яндекс Геокодера меня не устроила, поэтому результат его поисков использоваться не будет